In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix, precision_score, recall_score


# Define the path to the zip files
train_zip_path = r'C:\Users\Ryan\OneDrive\Documents\Computer Vision\CV Discussion3\train.zip'
test_zip_path = r'C:\Users\Ryan\OneDrive\Documents\Computer Vision\CV Discussion3\test.zip'

# Define the extraction directories
train_dir = r'C:\Users\Ryan\OneDrive\Documents\Computer Vision\CV Discussion3\train_data'

# Extract the training zip file
with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall(train_dir)


print("Extraction complete!")

Extraction complete!


In [10]:


# Define the correct extraction directory and subdirectory
train_dir = r'C:\Users\Ryan\OneDrive\Documents\Computer Vision\CV Discussion3\train_data\train'

# Initialize lists for file paths and labels
file_paths = []
labels = []

# Iterate through the files in the subdirectory and label them
for filename in os.listdir(train_dir):
    if filename.lower().startswith('cat'):
        labels.append(0)
        file_paths.append(os.path.join(train_dir, filename))
    elif filename.lower().startswith('dog'):
        labels.append(1)
        file_paths.append(os.path.join(train_dir, filename))

# Check if file_paths and labels have been populated
print(f"Number of file paths: {len(file_paths)}")
print(f"Number of labels: {len(labels)}")

# Create DataFrame if there are data
if file_paths and labels:
    df = pd.DataFrame({
        'filename': file_paths,
        'class': labels
    })

    print(f"DataFrame shape: {df.shape}")
    print(df.head())
else:
    print("No images found or labels not assigned correctly.")

Number of file paths: 25000
Number of labels: 25000
DataFrame shape: (25000, 2)
                                            filename  class
0  C:\Users\Ryan\OneDrive\Documents\Computer Visi...      0
1  C:\Users\Ryan\OneDrive\Documents\Computer Visi...      0
2  C:\Users\Ryan\OneDrive\Documents\Computer Visi...      0
3  C:\Users\Ryan\OneDrive\Documents\Computer Visi...      0
4  C:\Users\Ryan\OneDrive\Documents\Computer Visi...      0


In [11]:

# Convert numeric labels to strings
df['class'] = df['class'].map({0: 'cat', 1: 'dog'})

# Verify the changes
print(df.head())
print(df['class'].value_counts())

                                            filename class
0  C:\Users\Ryan\OneDrive\Documents\Computer Visi...   cat
1  C:\Users\Ryan\OneDrive\Documents\Computer Visi...   cat
2  C:\Users\Ryan\OneDrive\Documents\Computer Visi...   cat
3  C:\Users\Ryan\OneDrive\Documents\Computer Visi...   cat
4  C:\Users\Ryan\OneDrive\Documents\Computer Visi...   cat
class
cat    12500
dog    12500
Name: count, dtype: int64


In [14]:

# Define image size and batch size
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Create ImageDataGenerator instances
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 50,
    zoom_range = .2,
    horizontal_flip = True,
    brightness_range = [.5, 1.7],
    validation_split=0.2
)

# Training generator
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=None,  # Use None since file paths are absolute
    x_col='filename',
    y_col='class',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle = True
)

# Validation generator
validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=None,  # Use None since file paths are absolute
    x_col='filename',
    y_col='class',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle = True
)

print("Custom data generators created!")

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Custom data generators created!


In [16]:

# Load the pre-trained MobileNetV2 model
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',  # Load weights pre-trained on ImageNet
    include_top=False,  # Exclude the final dense layers
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)  # Input shape
)

# Freeze the base model
base_model.trainable = False

# Build the new model on top of the pre-trained model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Reduce dimensions from (H, W, C) to (C)
    layers.Dense(1024, activation='relu'),  # Add a dense layer
    layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Log loss function
    metrics=['accuracy']
)

In [17]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,  # Corrected variable name
    epochs=10
)

Epoch 1/10


C:\Users\Ryan\anaconda3\envs\computer_vision\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 1649s 3s/step - accuracy: 0.9364 - loss: 0.1637 - val_accuracy: 0.9168 - val_loss: 0.1867
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1526s 2s/step - accuracy: 0.9609 - loss: 0.0981 - val_accuracy: 0.9804 - val_loss: 0.0569
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1546s 2s/step - accuracy: 0.9620 - loss: 0.0909 - val_accuracy: 0.9562 - val_loss: 0.1058
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1503s 2s/step - accuracy: 0.9652 - loss: 0.0871 - val_accuracy: 0.9504 - val_loss: 0.1207
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1543s 2s/step - accuracy: 0.9651 - loss: 0.0856 - val_accuracy: 0.9632 - val_loss: 0.0871
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1564s 2s/step - accuracy: 0.9678 - loss: 0.0787 - val_accuracy: 0.9436 - val_loss: 0.1335
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1552s 2s/step - accuracy: 0.9684 - loss: 0.0759 - val_accuracy: 0.9628 - val_loss: 0.0998
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1593s 2s/step - accuracy: 0.9705 - loss: 0.0691 - val_accurac

In [25]:
test_dir = r'C:\Users\Ryan\OneDrive\Documents\Computer Vision\CV Discussion3\test_data\test'

# List all test images with valid extensions
test_image_files = [f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Create DataFrame with a 'filename' column
test_df = pd.DataFrame({
    'filename': [os.path.join(test_dir, f) for f in test_image_files]  # Full paths to test images
})

# Debug: Print the DataFrame to ensure it's correct
print("DataFrame Columns:", test_df.columns)
print(test_df.head())  # Check the first few rows to verify the filenames

DataFrame Columns: Index(['filename'], dtype='object')
                                            filename
0  C:\Users\Ryan\OneDrive\Documents\Computer Visi...
1  C:\Users\Ryan\OneDrive\Documents\Computer Visi...
2  C:\Users\Ryan\OneDrive\Documents\Computer Visi...
3  C:\Users\Ryan\OneDrive\Documents\Computer Visi...
4  C:\Users\Ryan\OneDrive\Documents\Computer Visi...


In [35]:

# Define the ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)

# Test data generator
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,  # Use None if file paths are absolute
    x_col='filename',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=1,  # Typically batch size of 1 for predictions
    class_mode=None,  # No labels for test data
    shuffle=False  # Ensure the order is maintained for results
)

# Make predictions
predictions = model.predict(test_generator)

# Convert predictions to probabilities
predicted_probabilities = predictions.flatten()

# Convert probabilities to binary class predictions
predicted_classes = (predicted_probabilities > 0.5).astype(int)


Found 12500 validated image filenames.
    2/12500 ━━━━━━━━━━━━━━━━━━━━ 15:13 73ms/step 

C:\Users\Ryan\anaconda3\envs\computer_vision\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12500/12500 ━━━━━━━━━━━━━━━━━━━━ 651s 52ms/step


In [36]:
# Create a DataFrame with IDs and predictions
results_df = pd.DataFrame({
    'ID': [os.path.splitext(os.path.basename(f))[0] for f in test_image_files],  # IDs from file names
    'label': predicted_classes  # 1 for dog, 0 for cat
})

# Save results to a CSV file
results_df.to_csv('cnn_submission.csv', index=False)
print('Predictions saved to submission.csv')

Predictions saved to submission.csv


In [39]:
#Create a datagenerator for all the training data for the confusion matrix
all_data_gen = ImageDataGenerator(rescale = 1./255)

train_full_generator = all_data_gen.flow_from_dataframe(
    dataframe=df,
    directory=None,  # Use None since file paths are absolute
    x_col='filename',
    y_col='class',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size= 1,
    class_mode='binary',
    shuffle = False
)

Found 25000 validated image filenames belonging to 2 classes.


C:\Users\Ryan\anaconda3\envs\computer_vision\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25000/25000 ━━━━━━━━━━━━━━━━━━━━ 1357s 54ms/step


ValueError: Found input variables with inconsistent numbers of samples: [20000, 25000]

In [43]:
# Predict on the entire training data
train_predictions = model.predict(train_full_generator)
train_predicted_labels = np.where(train_predictions > 0.5, 1, 0).flatten()  # Convert probabilities to binary class labels

# Get true labels from the full training generator
train_true_labels = train_full_generator.classes

# Generate the confusion matrix
conf_matrix_train = confusion_matrix(train_true_labels, train_predicted_labels)
precision_train = precision_score(train_true_labels, train_predicted_labels)
recall_train = recall_score(train_true_labels, train_predicted_labels)

# Print results
print('Confusion Matrix for Training Data:')
print(conf_matrix_train)
print(f'Precision: {precision_train:.2f}')
print(f'Recall: {recall_train:.2f}')



25000/25000 ━━━━━━━━━━━━━━━━━━━━ 1481s 59ms/step
Confusion Matrix for Training Data:
[[12190   310]
 [   73 12427]]
Precision: 0.98
Recall: 0.99
